---
© 2025 KR-Labs. All rights reserved.  
KR-Labs™ is a trademark of Quipu Research Labs, LLC, a subsidiary of Sudiata Giddasira, Inc.

SPDX-License-Identifier: Apache-2.0
---

# Tutorial 4: Regional Economic Analysis

**Author:** KRL Model Zoo Team  
**Affiliation:** KR-Labs  
**Version:** v1.0  
**Date:** October 25, 2025  
**License:** Apache 2.0  
**Tier:** Open-Source (Tier 1-3)

---

## Tutorial Overview

This tutorial demonstrates **regional economic analysis** using Location Quotient and Shift-Share methods to identify industry specialization and decompose employment changes across regions.

**Models Covered:**
- Location Quotient - Industry specialization analysis
- Shift-Share - Employment change decomposition

**Dataset:** Synthetic regional industry employment data (10 regions, 8 industries)

### Learning Objectives

By the end of this tutorial, you will be able to:
1. Calculate location quotients to identify regional specialization
2. Perform shift-share analysis to decompose employment changes
3. Identify competitive advantages and industry clusters
4. Inform economic development policy and strategy

### Prerequisites

- Understanding of regional economics concepts
- Familiarity with employment and industry data
- Basic knowledge of economic development policy

**Estimated Time:** 30-40 minutes

---

## Business Applications

Regional analysis is essential for:

- **Economic Development Planning:** Identify growth industries and clusters
- **Workforce Development:** Align training programs with regional needs
- **Policy Assessment:** Evaluate impact of economic development policies
- **Site Selection:** Identify optimal locations for business expansion

---

## Data Provenance

**Source:** Synthetic data generated for educational purposes  
**Characteristics:**
- 10 regions with distinct specialization patterns
- 8 industries (Manufacturing, Technology, Finance, Healthcare, Retail, Agriculture, Construction, Other)
- Employment, establishment, and wage data
- Specialized clusters: Tech Hub, Manufacturing Center, Financial Center

**Real-world Equivalents:**
- U.S. Census Bureau County Business Patterns
- Bureau of Labor Statistics Quarterly Census of Employment and Wages (QCEW)
- Regional economic development data

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Import KRL Model Zoo regional models
from krl_models.regional import LocationQuotientModel, ShiftShareModel
from krl_core import ModelMeta

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Load Regional Employment Data

In [ ]:
# Load employment data
df = pd.read_csv('../data/regional_industry.csv')

print(f"Dataset shape: {df.shape}")
print(f"Regions: {df['region'].nunique()}")
print(f"Industries: {df['industry'].nunique()}")
print(f"\nSample data:")
df.head(10)

In [ ]:
# Summary statistics by region
region_summary = df.groupby('region')['employment'].sum().sort_values(ascending=False)
print("\nTotal Employment by Region:")
print(region_summary)

# Summary by industry
industry_summary = df.groupby('industry')['employment'].sum().sort_values(ascending=False)
print("\nTotal Employment by Industry:")
print(industry_summary)

In [ ]:
# Visualize employment distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# By region
region_summary.plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Total Employment by Region', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Employment')
axes[0].grid(True, alpha=0.3)

# By industry
industry_summary.plot(kind='barh', ax=axes[1], color='seagreen')
axes[1].set_title('Total Employment by Industry', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Employment')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Model 1: Location Quotient (LQ) Analysis

Location Quotient measures regional specialization in an industry relative to the national average.

**Formula:**
```
LQ = (Regional Employment in Industry i / Total Regional Employment) / 
     (National Employment in Industry i / Total National Employment)
```

**Interpretation:**
- LQ > 1.25: High specialization (export-oriented)
- 0.75 < LQ < 1.25: Similar to national average
- LQ < 0.75: Under-represented (imports from other regions)

In [ ]:
# Configure Location Quotient model
lq_params = {
    'region_col': 'region',
    'industry_col': 'industry',
    'employment_col': 'employment'
}

meta_lq = ModelMeta(
    name="Regional_LQ",
    version="1.0",
    author="Tutorial",
    description="Location Quotient analysis for industry specialization"
)

# Fit LQ model
lq_model = LocationQuotientModel(df, lq_params, meta_lq)
lq_result = lq_model.fit()

print("Location Quotient model fitted successfully!")
print(f"\nNumber of region-industry combinations analyzed: {len(lq_result.payload['location_quotients'])}")

In [ ]:
# Extract LQ results
lq_df = lq_result.payload['location_quotients']
print("\nLocation Quotient Results (Sample):")
print(lq_df.head(15))

In [ ]:
# Find highly specialized industries by region
print("\nHighly Specialized Industries (LQ > 1.25) by Region:")
print("=" * 70)

for region in df['region'].unique()[:5]:  # Show first 5 regions
    region_lqs = lq_df[lq_df['region'] == region].sort_values('location_quotient', ascending=False)
    specialized = region_lqs[region_lqs['location_quotient'] > 1.25]
    
    print(f"\n{region}:")
    if len(specialized) > 0:
        for _, row in specialized.head(3).iterrows():
            print(f"  - {row['industry']}: LQ = {row['location_quotient']:.2f}")
    else:
        print("  No highly specialized industries")

In [ ]:
# Visualize LQ for a specific region
sample_region = df['region'].unique()[0]
region_lqs = lq_df[lq_df['region'] == sample_region].sort_values('location_quotient', ascending=False)

plt.figure(figsize=(12, 8))
colors = ['green' if lq > 1.25 else 'orange' if lq > 0.75 else 'red' 
          for lq in region_lqs['location_quotient']]

plt.barh(range(len(region_lqs)), region_lqs['location_quotient'], color=colors)
plt.yticks(range(len(region_lqs)), region_lqs['industry'])
plt.axvline(x=1.0, color='black', linestyle='--', linewidth=2, label='National Average')
plt.axvline(x=1.25, color='green', linestyle=':', linewidth=1.5, alpha=0.7, label='Specialization Threshold')
plt.axvline(x=0.75, color='red', linestyle=':', linewidth=1.5, alpha=0.7, label='Under-representation Threshold')
plt.xlabel('Location Quotient')
plt.title(f'Location Quotients - {sample_region}', fontsize=16, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print(f"\nGreen: Export-oriented (LQ > 1.25)")
print(f"Orange: Similar to national average (0.75-1.25)")
print(f"Red: Under-represented (LQ < 0.75)")

## Model 2: Shift-Share Analysis

Shift-Share decomposes employment change into three components:

1. **National Growth Effect:** Change if region grew at national rate
2. **Industry Mix Effect:** Advantage/disadvantage from industry composition
3. **Regional Shift Effect:** Competitive advantage/disadvantage

**Interpretation:**
- Positive Regional Shift: Region is competitive in that industry
- Negative Regional Shift: Region is losing competitiveness

In [ ]:
# For shift-share, we need data from two time periods
# Let's simulate time period data
base_year_data = df.copy()
base_year_data['year'] = 2020

# Create end year data with growth
end_year_data = df.copy()
end_year_data['employment'] = end_year_data['employment'] * np.random.uniform(0.95, 1.15, len(end_year_data))
end_year_data['year'] = 2023

# Combine
shift_share_data = pd.concat([base_year_data, end_year_data], ignore_index=True)

print(f"Shift-Share dataset shape: {shift_share_data.shape}")
print("\nSample data:")
shift_share_data.head()

In [ ]:
# Configure Shift-Share model
ss_params = {
    'region_col': 'region',
    'industry_col': 'industry',
    'employment_col': 'employment',
    'time_col': 'year',
    'base_year': 2020,
    'end_year': 2023
}

meta_ss = ModelMeta(
    name="Regional_ShiftShare",
    version="1.0",
    author="Tutorial",
    description="Shift-Share decomposition of employment change"
)

# Fit Shift-Share model
ss_model = ShiftShareModel(shift_share_data, ss_params, meta_ss)
ss_result = ss_model.fit()

print("Shift-Share model fitted successfully!")

In [ ]:
# Extract shift-share results
ss_df = ss_result.payload['shift_share_components']
print("\nShift-Share Components (Sample):")
print(ss_df.head(15))

In [ ]:
# Analyze a specific region's employment dynamics
sample_region = df['region'].unique()[0]
region_ss = ss_df[ss_df['region'] == sample_region].copy()

print(f"\n{sample_region} Employment Change Analysis:")
print("=" * 70)
print(f"\nTotal employment change: {region_ss['total_change'].sum():.0f}")
print(f"\nBreakdown:")
print(f"  National Growth:    {region_ss['national_growth'].sum():.0f}")
print(f"  Industry Mix:       {region_ss['industry_mix'].sum():.0f}")
print(f"  Regional Shift:     {region_ss['regional_shift'].sum():.0f}")

# Show top performers and losers
region_ss_sorted = region_ss.sort_values('regional_shift', ascending=False)

print(f"\nMost Competitive Industries (Positive Regional Shift):")
for _, row in region_ss_sorted.head(3).iterrows():
    print(f"  {row['industry']}: +{row['regional_shift']:.0f}")

print(f"\nLeast Competitive Industries (Negative Regional Shift):")
for _, row in region_ss_sorted.tail(3).iterrows():
    print(f"  {row['industry']}: {row['regional_shift']:.0f}")

In [ ]:
# Visualize shift-share components for region
components = ['national_growth', 'industry_mix', 'regional_shift']
component_totals = [region_ss[comp].sum() for comp in components]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart of components
axes[0].bar(components, component_totals, 
            color=['steelblue', 'orange', 'green'], edgecolor='black', linewidth=1.5)
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.8)
axes[0].set_title(f'{sample_region}: Shift-Share Components', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Employment Change')
axes[0].set_xticklabels(['National\nGrowth', 'Industry\nMix', 'Regional\nShift'])
axes[0].grid(True, alpha=0.3, axis='y')

# Industry-level regional shift
industry_shift = region_ss.sort_values('regional_shift', ascending=True)
colors = ['green' if x > 0 else 'red' for x in industry_shift['regional_shift']]
axes[1].barh(range(len(industry_shift)), industry_shift['regional_shift'], color=colors)
axes[1].set_yticks(range(len(industry_shift)))
axes[1].set_yticklabels(industry_shift['industry'])
axes[1].axvline(x=0, color='black', linestyle='-', linewidth=0.8)
axes[1].set_title(f'{sample_region}: Regional Shift by Industry', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Regional Shift Effect')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## Combined Analysis: LQ and Shift-Share

Combine both analyses to identify strategic industries.

In [ ]:
# Merge LQ and Shift-Share results
combined = region_ss.merge(
    lq_df[lq_df['region'] == sample_region][['industry', 'location_quotient']], 
    on='industry'
)

# Create strategic quadrant
combined['strategy'] = 'Unknown'
combined.loc[(combined['location_quotient'] > 1.25) & (combined['regional_shift'] > 0), 'strategy'] = 'Stars'
combined.loc[(combined['location_quotient'] > 1.25) & (combined['regional_shift'] <= 0), 'strategy'] = 'Mature'
combined.loc[(combined['location_quotient'] <= 1.25) & (combined['regional_shift'] > 0), 'strategy'] = 'Emerging'
combined.loc[(combined['location_quotient'] <= 1.25) & (combined['regional_shift'] <= 0), 'strategy'] = 'Declining'

print(f"\n{sample_region} Strategic Industry Classification:")
print("=" * 70)

for strategy in ['Stars', 'Emerging', 'Mature', 'Declining']:
    industries = combined[combined['strategy'] == strategy]['industry'].tolist()
    print(f"\n{strategy}:")
    if len(industries) > 0:
        for ind in industries[:5]:
            print(f"  - {ind}")
    else:
        print("  None")

In [ ]:
# Strategic quadrant visualization
plt.figure(figsize=(12, 8))

colors_map = {
    'Stars': 'green',
    'Emerging': 'blue',
    'Mature': 'orange',
    'Declining': 'red'
}

for strategy in combined['strategy'].unique():
    subset = combined[combined['strategy'] == strategy]
    plt.scatter(subset['location_quotient'], subset['regional_shift'], 
                label=strategy, s=100, alpha=0.7, color=colors_map.get(strategy, 'gray'))

# Add quadrant lines
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.axvline(x=1.25, color='black', linestyle='--', linewidth=1)

# Add labels
plt.xlabel('Location Quotient (Specialization)', fontsize=12)
plt.ylabel('Regional Shift (Competitiveness)', fontsize=12)
plt.title(f'{sample_region}: Strategic Industry Portfolio', fontsize=16, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Annotate some industries
for _, row in combined.iterrows():
    if abs(row['regional_shift']) > combined['regional_shift'].std() or row['location_quotient'] > 1.5:
        plt.annotate(row['industry'], 
                    (row['location_quotient'], row['regional_shift']),
                    fontsize=8, alpha=0.7)

plt.tight_layout()
plt.show()

print("\nStrategic Recommendations:")
print("- Stars: Maintain competitive advantage, support growth")
print("- Emerging: Invest in development, build infrastructure")
print("- Mature: Diversify, improve productivity")
print("- Declining: Exit or restructure, retrain workforce")

## Practical Applications

### 1. Economic Development Strategy

In [ ]:
# Identify target industries for development
print("Recommended Focus Industries for Economic Development:")
print("=" * 70)

# Stars: Already strong, maintain
stars = combined[combined['strategy'] == 'Stars'].sort_values('regional_shift', ascending=False)
print("\nPriority 1 - Maintain Competitive Advantage (Stars):")
for _, row in stars.head(3).iterrows():
    print(f"  {row['industry']}: LQ={row['location_quotient']:.2f}, Shift={row['regional_shift']:.0f}")

# Emerging: Growing strength
emerging = combined[combined['strategy'] == 'Emerging'].sort_values('regional_shift', ascending=False)
print("\nPriority 2 - Develop Growth Sectors (Emerging):")
for _, row in emerging.head(3).iterrows():
    print(f"  {row['industry']}: LQ={row['location_quotient']:.2f}, Shift={row['regional_shift']:.0f}")

### 2. Workforce Planning

In [ ]:
# Identify industries needing workforce training
print("\nWorkforce Training Priorities:")
print("=" * 70)

# Growing industries (positive shift)
growing = combined[combined['regional_shift'] > 0].sort_values('regional_shift', ascending=False)
print("\nSkill Development Needed (Growing Industries):")
for _, row in growing.head(5).iterrows():
    print(f"  {row['industry']}: Expected growth of {row['regional_shift']:.0f} jobs")

# Declining industries (negative shift)
declining_jobs = combined[combined['regional_shift'] < 0].sort_values('regional_shift')
print("\nWorkforce Transition Support Needed (Declining Industries):")
for _, row in declining_jobs.head(5).iterrows():
    print(f"  {row['industry']}: Expected loss of {abs(row['regional_shift']):.0f} jobs")

## Key Takeaways

1. **Location Quotient (LQ):**
   - Measures regional specialization vs national average
   - LQ > 1.25: Export-oriented, regional strength
   - LQ < 0.75: Import-dependent, regional weakness
   - Use for identifying industry clusters

2. **Shift-Share Analysis:**
   - Decomposes employment change into 3 components
   - National Growth: Overall economic expansion
   - Industry Mix: Effect of industry composition
   - Regional Shift: Local competitiveness
   - Positive Regional Shift = competitive advantage

3. **Strategic Framework:**
   - **Stars** (High LQ, Positive Shift): Maintain and support
   - **Emerging** (Low LQ, Positive Shift): Develop and grow
   - **Mature** (High LQ, Negative Shift): Diversify and modernize
   - **Declining** (Low LQ, Negative Shift): Exit or restructure

4. **Policy Applications:**
   - Economic development planning
   - Workforce training priorities
   - Infrastructure investment decisions
   - Industry recruitment targeting

5. **Data Requirements:**
   - Regional and national employment by industry
   - Multiple time periods for shift-share
   - Consistent industry classifications

## Next Steps

- Analyze multiple regions simultaneously
- Add more time periods for trend analysis
- Include wage data for quality analysis
- Compare shift-share results across regions
- Integrate with economic forecasting models
- Use results to inform policy decisions

In [ ]:
from datetime import datetime
from pathlib import Path
import json

# Create output directory
output_dir = Path('../outputs') / f'regional_analysis_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Exporting results to: {output_dir}\n")

# Export location quotient results
lq_results.to_csv(output_dir / 'location_quotients.csv', index=False)
print(" Exported location quotient results")

# Export shift-share results
ss_results.to_csv(output_dir / 'shift_share_analysis.csv', index=False)
print(" Exported shift-share results")

# Export metadata
metadata = {
    "tutorial": "04_regional_analysis.ipynb",
    "version": "v1.0",
    "execution_date": datetime.now().isoformat(),
    "models": ["Location Quotient", "Shift-Share"],
    "dataset": {
        "name": "regional_industry_sample.csv",
        "regions": len(df['region'].unique()),
        "industries": len(df['industry'].unique()),
        "records": len(df)
    },
    "reproducibility": {
        "random_seed": None,
        "python_version": "3.9+",
        "required_packages": ["krl-model-zoo", "pandas", "numpy", "matplotlib"]
    }
}

with open(output_dir / 'execution_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print(" Exported execution metadata")

print(f"\n{'='*60}")
print("EXPORT COMPLETE")
print(f"{'='*60}")

## Responsible Use & Limitations

### Ethical Considerations

1. **Data Privacy:**
   - This analysis uses aggregated regional employment data
   - Real applications should use publicly available Census/BLS data
   - Never use confidential business-level data without proper authorization

2. **Bias & Fairness:**
   - Regional specialization reflects historical economic patterns
   - Consider equity and inclusion when making policy recommendations
   - Avoid reinforcing historical disadvantages or inequalities
   - Ensure economic development benefits all communities

3. **Limitations:**
   - Synthetic data for demonstration purposes only
   - Real analysis requires domain expertise and local knowledge
   - Location quotients are descriptive, not causal measures
   - Shift-share assumes stable industry mix over time
   - Results sensitive to industry classification and geographic boundaries

4. **Recommended Use Cases:**
   -  Educational purposes and learning
   -  Strategic economic development planning
   -  Industry cluster identification
   -  Workforce development program design
   -  High-stakes policy decisions without expert consultation
   -  Business location decisions without site visits
   -  Predictions of future economic outcomes

5. **Model Assumptions:**
   - Location Quotient assumes uniform industry productivity across regions
   - Shift-Share assumes constant competitive effects within periods
   - Both methods sensitive to industry classification systems
   - Regional boundaries significantly affect results

### Best Practices

- Validate results with local economic development experts
- Use multiple data sources and methods for robust analysis
- Consider qualitative factors (infrastructure, workforce, quality of life)
- Update analysis regularly as economic conditions change
- Engage stakeholders in interpreting and applying results
- Document all assumptions, data sources, and limitations
- Consider spatial relationships and spillover effects

For questions about responsible use: info@krlabs.dev

## References

1. **Isard, W.** (1960). *Methods of Regional Analysis: An Introduction to Regional Science*. MIT Press.

2. **Klosterman, R. E.** (1990). Community analysis and planning techniques. Rowman & Littlefield.

3. **Blair, J. P., & Carroll, M. C.** (2009). *Local Economic Development: Analysis, Practices, and Globalization* (2nd ed.). SAGE Publications.

4. **Esteban-Marquillas, J. M.** (1972). A reinterpretation of shift-share analysis. *Regional and Urban Economics*, 2(3), 249-255.

5. **U.S. Census Bureau.** (2024). *County Business Patterns*. https://www.census.gov/programs-surveys/cbp.html

6. **U.S. Bureau of Labor Statistics.** (2024). *Quarterly Census of Employment and Wages*. https://www.bls.gov/cew/

---

## Citation

To cite this tutorial:

```bibtex
@misc{krl_regional_analysis_2025,
  title = {Tutorial 4: Regional Economic Analysis},
  author = {KRL Model Zoo Team},
  year = {2025},
  publisher = {KR-Labs},
  url = {https://github.com/KR-Labs/krl-model-zoo},
  note = {Tutorial from KRL Model Zoo v1.0.0}
}
```

To cite KRL Model Zoo:

```bibtex
@software{krl_model_zoo_2025,
  title = {KRL Model Zoo: Production-Grade Models for Socioeconomic Analysis},
  author = {KR-Labs},
  year = {2025},
  url = {https://github.com/KR-Labs/krl-model-zoo},
  version = {1.0.0},
  license = {Apache-2.0}
}
```

---

<div style="text-align: center; padding: 20px 0; border-top: 2px solid #333; margin-top: 40px;">
  <p style="font-size: 14px; font-weight: bold; margin-bottom: 5px;">
    KR-Labs | Data-Driven Economic Analysis
  </p>
  <p style="font-size: 12px; color: #666; margin: 5px 0;">
    Contact: <a href="mailto:info@krlabs.dev">info@krlabs.dev</a>
  </p>
  <p style="font-size: 11px; color: #666; margin: 5px 0;">
    © 2025 KR-Labs. All rights reserved.<br>
    <strong>KR-Labs™</strong> is a trademark of Quipu Research Labs, LLC, a subsidiary of Sudiata Giddasira, Inc.
  </p>
  <p style="font-size: 11px; color: #666; margin: 5px 0;">
    <a href="https://www.apache.org/licenses/LICENSE-2.0" target="_blank">Apache 2.0 License</a> | 
    <a href="https://github.com/KR-Labs/krl-model-zoo" target="_blank">GitHub Repository</a>
  </p>
</div>

## Export Results & Reproducibility

This section exports analysis results and metadata for reproducibility.